In [56]:
from Crypto.Util.number import *
import hashlib
from pwn import xor
import random

# User information
id = "HappyHacker22"
pw = "123456"

# Initialization phase
delta_T = 1
while(True):
    q = getPrime(128)
    if (isPrime(2*q + 1)):
        p = 2*q + 1
        break
Fp = GF(q, modulus = "primitive")
g = Fp.gen()

# Choose md5 as a secure one way hash function
sk = Fp.random_element()
pk = g**sk
id_table = []

print("Initialisation phase : ")
print("p =", p)
print("q =", q)
print("Finite field generator g =", g)
print("The hash function is fixed to be : sha256")
print("Secret key (kept secret) sk =", sk)
print("Public key pk =", pk)
print("Maximum transmission delay = ", delta_T)

print("\nUser information :")
print("Username :", id)
print("Password :", pw)

# Register phase
h1 = b""
h2 = b""
h1 = hashlib.sha256((id.encode() + long_to_bytes(sk))).digest()
h2 = hashlib.sha256(pw.encode()).digest()
y = xor(h1, h2)
id_table.append(bytes_to_long(id.encode()))

print("\nRegister phase :")
print("Public key pk =", pk)
print("y_i =", bytes.hex(y))
print("H : sha256")
print("p =", p)
print("q =", q)

# Login and authentication phase

# User side
a = Fp.random_element()
y_dot = xor(y, hashlib.sha256(pw.encode()).digest())
c = g**a
d = pk**a
T_1 = 10

v = hashlib.sha256(id.encode() + y_dot + long_to_bytes(c) + long_to_bytes(d) + long_to_bytes(T_1)).digest()

# Server side
T_2 = 13
assert (T_2 > T_1) & (T_2 - T_1 > delta_T)
Y_dot = hashlib.sha256(id.encode() + long_to_bytes(sk)).digest()
D = c**sk
assert D == d

V = hashlib.sha256(id.encode() + Y_dot + long_to_bytes(c) + long_to_bytes(d) + long_to_bytes(T_1)).digest()
assert v == V

T_3 = 15
m = hashlib.sha256(id.encode() + long_to_bytes(D) + long_to_bytes(T_3)).digest()

# User side
assert (T_3 > T_2) & (T_3 - T_2 > delta_T)
M = hashlib.sha256(id.encode() + long_to_bytes(d) + long_to_bytes(T_3)).digest()

print("\nLogin and authentication phase : ")
if (m == M):
    print("The server S has successfully authenticated to the user U")
else:
    print("Authentication failed")
    
# Session keys calculation
session_key_U = hashlib.sha256(long_to_bytes(d)).hexdigest()
session_key_S = hashlib.sha256(long_to_bytes(D)).hexdigest()

print("User session key :", session_key_U)
print("Server session key :", session_key_S)

    





Initialisation phase : 
p = 638446356312588070731828207420759923219
q = 319223178156294035365914103710379961609
Finite field generator g = 3
The hash function is fixed to be : sha256
Secret key (kept secret) sk = 242854333492640340872182507813667603515
Public key pk = 53777379820330495502234739564145489682
Maximum transmission delay =  1

User information :
Username : HappyHacker22
Password : 123456

Register phase :
Public key pk = 53777379820330495502234739564145489682
y_i = 3870e91e16a1e82c874e335ff9408d65a0f4f136f5b22b89e94b0bd193f89b69
H : sha256
p = 638446356312588070731828207420759923219
q = 319223178156294035365914103710379961609

Login and authentication phase : 
The server S has successfully authenticated to the user U
User session key : 761d68b62ffc65f483438fdaa423036fc0aa1d8028aa2ad7b436463c55b4aa44
Server session key : 761d68b62ffc65f483438fdaa423036fc0aa1d8028aa2ad7b436463c55b4aa44
